# Prediction of store sales

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


## Load Data

In [2]:
import pandas as pd
import sys
import os
sys.path.append('/content/gdrive/MyDrive')
# from data_processing import *
# from eda import promo_distr as prd
# from eda import *
# from plots import *
import prediction as dp
import matplotlib.pyplot as plt
%matplotlib
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

Using matplotlib backend: <object object at 0x7983272dbb90>


In [3]:
from scipy.stats import chi2_contingency, ttest_ind
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import mutual_info_classif

## preprocessing

It is important to process the data into a format where it can be fed to a machine learning
model. This typically means converting all non-numeric columns to numeric, handling NaN
values, and generating new features from already existing features.

In [5]:
# df = pd.read_csv('../data/cleaned.csv')
df = pd.read_csv('/content/gdrive/MyDrive/cleaned.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

pd.set_option('display.max_columns', None)

In [ ]:
df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,SalePerCustomer,is_holiday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1,2015,7,31,31,9.482883,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2015-07-31,2,5,6064,625,1,1,0,1,2015,7,31,31,9.702400,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2015-07-31,3,5,8314,821,1,1,0,1,2015,7,31,31,10.126675,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2015-07-31,4,5,13995,1498,1,1,0,1,2015,7,31,31,9.342457,0,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
2015-07-31,5,5,4822,559,1,1,0,1,2015,7,31,31,8.626118,0,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2015-07-31 to 2013-01-01
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Sales                      1017209 non-null  int64  
 3   Customers                  1017209 non-null  int64  
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  object 
 7   SchoolHoliday              1017209 non-null  int64  
 8   Year                       1017209 non-null  int64  
 9   Month                      1017209 non-null  int64  
 10  Day                        1017209 non-null  int64  
 11  WeekOfYear                 1017209 non-null  int64  
 12  SalePerCustomer            844340 non-null   float64
 1

**Handling missing values in sales per customer**

- Have null value because of the day where specific store has no customers. I am going to replace it with zero which is zero if sales or customers value is zero

In [6]:
# Replace sales per customer null values with zero
df['SalePerCustomer'].fillna(0, inplace=True)

In [ ]:
df.isnull().sum()

Store                        0
DayOfWeek                    0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
Year                         0
Month                        0
Day                          0
WeekOfYear                   0
SalePerCustomer              0
is_holiday                   0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [ ]:
df.shape

(1017209, 23)

In [ ]:
df.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear',
       'SalePerCustomer', 'is_holiday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

## Building models with sklearn pipelines

In [7]:
categorical_feature = ['StateHoliday','StoreType', 'Assortment', 'PromoInterval']
num_cat = ['Customers', 'Open', 'Promo','DayOfWeek', 'CompetitionDistance','Year', 'Month', 'Day', 'WeekOfYear', 'Promo2']
numerical_feature = df.select_dtypes(include=['int', 'float']).columns.tolist()

In [8]:
X = df[numerical_feature]
y = df['Sales']

In [ ]:
X.shape

(1017209, 19)

In [9]:
# Encode and scale the data

# Converting to Categorical
def to_categorical(df, columns_to_convert):
    # Convert to categorical data type
    for column in columns_to_convert:
        df[column] = df[column].astype('category')
to_categorical(df, categorical_feature)

numerical_feature = df.select_dtypes(include=['int', 'float']).columns.tolist()
numerical_feature.remove('Sales')
all_feature = categorical_feature + num_cat
df_feat = df[all_feature]
print(df_feat.shape)


(1017209, 14)


In [ ]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2015-07-31 to 2013-01-01
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   StateHoliday         1017209 non-null  category
 1   StoreType            1017209 non-null  category
 2   Assortment           1017209 non-null  category
 3   PromoInterval        1017209 non-null  category
 4   Customers            1017209 non-null  int64   
 5   Open                 1017209 non-null  int64   
 6   Promo                1017209 non-null  int64   
 7   DayOfWeek            1017209 non-null  int64   
 8   CompetitionDistance  1017209 non-null  float64 
 9   Year                 1017209 non-null  int64   
 10  Month                1017209 non-null  int64   
 11  Day                  1017209 non-null  int64   
 12  WeekOfYear           1017209 non-null  int64   
 13  Promo2               1017209 non-null  int64   
dtypes: category(4), flo

In [10]:
df_label = dp.encoder('oneHotEncoder', df_feat, categorical_feature)

In [ ]:
df_label.shape

(1017209, 22)

In [ ]:
df_label.head()

In [ ]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2015-07-31 to 2013-01-01
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Customers             1017209 non-null  int64  
 1   Open                  1017209 non-null  int64  
 2   Promo                 1017209 non-null  int64  
 3   DayOfWeek             1017209 non-null  int64  
 4   CompetitionDistance   1017209 non-null  float64
 5   Year                  1017209 non-null  int64  
 6   Month                 1017209 non-null  int64  
 7   Day                   1017209 non-null  int64  
 8   WeekOfYear            1017209 non-null  int64  
 9   Promo2                1017209 non-null  int64  
 10  ohe_0                 1017209 non-null  int8   
 11  ohe_a                 1017209 non-null  int8   
 12  ohe_b                 1017209 non-null  int8   
 13  ohe_c                 1017209 non-null  int8   
 14  ohe_b              

In [11]:
df_scaled = dp.scaler('standardScaler', df_label, num_cat)

In [ ]:
df_scaled.head()

,Customers,Open,Promo,DayOfWeek,CompetitionDistance,Year,Month,Day,WeekOfYear,Promo2,ohe_0,ohe_a,ohe_b,ohe_c,ohe_b,ohe_c,ohe_d,ohe_b,ohe_c,"ohe_Feb,May,Aug,Nov","ohe_Jan,Apr,Jul,Oct","ohe_Mar,Jun,Sept,Dec"
Date,,,,,,,,,,,,,,,,,,,,,,
2015-07-31,-0.168269,0.452399,1.273237,0.501484,-0.538740,1.502077,0.346724,1.740766,0.511626,-1.001128,0,0,0,0,0,1,0,0,0,0,0,0
2015-07-31,-0.017540,0.452399,1.273237,0.501484,-0.629567,1.502077,0.346724,1.740766,0.511626,0.998873,0,0,0,0,0,0,0,0,0,0,1,0
2015-07-31,0.404499,0.452399,1.273237,0.501484,1.129892,1.502077,0.346724,1.740766,0.511626,0.998873,0,0,0,0,0,0,0,0,0,0,1,0
2015-07-31,1.862258,0.452399,1.273237,0.501484,-0.623080,1.502077,0.346724,1.740766,0.511626,-1.001128,0,0,0,0,0,1,0,0,1,0,0,0
2015-07-31,-0.159656,0.452399,1.273237,0.501484,3.177404,1.502077,0.346724,1.740766,0.511626,-1.001128,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_scaled.shape

(1017209, 22)

In [12]:
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Split the data
X = df_scaled.copy()
y = df['Sales']
X_train, X_test, y_train, y_test = split_data(X, y)

In [13]:
# Before fiting the data into the model, lets find the best number of estimators for both models
def mae_score(estimator):
    model1 = RandomForestRegressor(n_estimators= estimator, random_state=0)
    val_score = -1 * cross_val_score(model1, X, y, cv=2, scoring='neg_mean_absolute_error')

    return val_score.mean()

In [14]:
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [15]:
rf_model = RandomForestRegressor(n_estimators= 100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

## Choose a loss function

In [16]:
  def evaluate_model(model, X_test, y_test):
    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return mae, mse, r2, y_pred

  models = ['Decision Tree', 'Random Forest']
  mae_scores, mse_scores, r2_scores = [], [], []
  # mae, mse, r2, _ = evaluate_model(dt_model, X_test, y_test)
  for model in [dt_model, rf_model]:
      mae, mse, r2, _ = evaluate_model(model, X_test, y_test)
      mae_scores.append(mae)
      mse_scores.append(mse)
      r2_scores.append(r2)


In [ ]:
 print('mae score for decision tree and random forest: ', mae_scores)
 print('mse score for decision tree and random forest: ', mse_scores) # 900, 180  mae
 print('accuracy for decision tree and random forest: ', r2_scores)

mae score for decision tree and random forest:  [419.14664376087535, 300.374460835275]
mse score for decision tree and random forest:  [515805.30164494057, 254640.0636612415]
accuracy for decision tree and random forest:  [0.9651218871604613, 0.9827815556654405]


## Serialize models

In [ ]:
import pickle
from datetime import datetime

# Serialize the model with time stamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"model_{timestamp}.pkl"

# Save the model to a pickle file
with open(model_filename, "wb") as file:
    pickle.dump(rf_model, file)
print(f"Model saved as {model_filename}")

## Building model with deep learning

In [13]:
#Further divide training dataset into train and validation dataset with an 90:10 split
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.1,random_state=42)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM


In [18]:
model = Sequential()
model.add(Dense(150,input_dim = 22,activation="relu"))
#The input_dim =44, since the width of the training data=44 (refer data engg section)
model.add(Dense(1,activation = "linear"))


In [19]:
#Configure the model
model.compile(optimizer='adam',loss="mean_absolute_error", metrics=["mean_absolute_error"])

In [20]:
#Train the model
model.fit(x_train.values,y_train.values, validation_data=(x_val,y_val),epochs=10,batch_size=64)

Epoch 1/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 3065.3494 - mean_absolute_error: 3065.3494 - val_loss: 763.7226 - val_mean_absolute_error: 763.7226
Epoch 2/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 756.4307 - mean_absolute_error: 756.4307 - val_loss: 731.5231 - val_mean_absolute_error: 731.5231
Epoch 3/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 728.6240 - mean_absolute_error: 728.6240 - val_loss: 712.1663 - val_mean_absolute_error: 712.1663
Epoch 4/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 711.7879 - mean_absolute_error: 711.7879 - val_loss: 702.5219 - val_mean_absolute_error: 702.5219
Epoch 5/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 702.5520 - mean_absolute_error: 702.5520 - val_loss: 694.5412 - val_mean_absolute_error: 694.5412
Epoch 6/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 695.2829 - mean_absolute_error: 695.2829 - val_loss: 684.9407 - val_mean_absolute_error: 684.9407
Epoch 7/10
11444/114

**Testing the Model Performance**

In [22]:
import numpy as np

#Use the model's evaluate method to predict and evaluate the test datasets
result = model.evaluate(np.asarray(X_test.values).astype('float32'), np.asarray(y_test.values).astype('float32'))

#Print the results
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

6358/6358 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 670.3521 - mean_absolute_error: 670.3521
Metric  loss : 669.6
Metric  compile_metrics : 669.6


**Improving the Model**
* In the following network, we have added two more layers with similar numbers of neurons
* We will update our loss function to mean squared error instead of MAE

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 22,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
history = model.fit(x_train,y_train, validation_data=(x_val,
y_val),epochs=10,batch_size=64)
#result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

- Lstm

In [28]:
# Convert the DataFrame to a NumPy array before reshaping
X_values = X.values
X = X_values.reshape(X_values.shape[0], X_values.shape[1], 1)

X_train, X_test, y_train, y_test = split_data(X, y)
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.1,random_state=42)

In [29]:
#Check the sizes of all newly created datasets
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of X_test:",X_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

Shape of x_train: (732390, 22, 1)
Shape of x_val: (81377, 22, 1)
Shape of X_test: (203442, 22, 1)
Shape of y_train: (732390,)
Shape of y_val: (81377,)
Shape of y_test: (203442,)


In [37]:
model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss="mean_absolute_error", metrics=["mean_absolute_error"]) # , metrics=['accuracy']

In [35]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val),batch_size=64)

Epoch 1/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 90s 8ms/step - loss: 2779.9636 - mean_absolute_error: 2779.9636 - val_loss: 917.3665 - val_mean_absolute_error: 917.3665
Epoch 2/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 145s 8ms/step - loss: 1106.0321 - mean_absolute_error: 1106.0321 - val_loss: 754.8845 - val_mean_absolute_error: 754.8845
Epoch 3/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 140s 8ms/step - loss: 1020.6355 - mean_absolute_error: 1020.6355 - val_loss: 742.1803 - val_mean_absolute_error: 742.1803
Epoch 4/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 140s 8ms/step - loss: 989.1786 - mean_absolute_error: 989.1786 - val_loss: 685.7711 - val_mean_absolute_error: 685.7711
Epoch 5/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 89s 8ms/step - loss: 958.7285 - mean_absolute_error: 958.7285 - val_loss: 694.7459 - val_mean_absolute_error: 694.7459
Epoch 6/10
11444/11444 ━━━━━━━━━━━━━━━━━━━━ 141s 8ms/step - loss: 943.8748 - mean_absolute_error: 943.8748 - val_loss: 678.3433 - val_mean_absolute_error: 678.3433
Epoch 7/10
1

In [ ]:
import pickle
from datetime import datetime

# Serialize the model with time stamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"model_{timestamp}.pkl"

# Save the model to a pickle file
with open(model_filename, "wb") as file:
    pickle.dump(model, file)
print(f"Model saved as {model_filename}")

In [39]:
#Use the model's evaluate method to predict and evaluate the test datasets
result = model.evaluate(X_test.astype('float32'), y_test.astype('float32'))

#Print the results
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

6358/6358 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 5772.2729 - mean_absolute_error: 5772.2729
Metric  loss : 5770.36
Metric  compile_metrics : 5770.36
